In [1]:
import numpy as np
import torch

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_sample_weight
from numpy import random

import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

import os
from pathlib import Path
home = str(Path.home())
base_path = os.path.join(home, 'workspace/multilid/data/extract/run_1/')

In [37]:
nor_fgsm = torch.load(os.path.join(base_path, "cifar10/wrn28-10/fgsm/lid/lid_normalos_8255.pt"))
adv_fgsm = torch.load("../data/extracted_characteristics/run_1/cif10/wrn28-10/fgsm/lid/lid_adverlos_8255.p")
nor_bim  = torch.load("../data/extracted_characteristics/run_1/cif10/wrn28-10/bim/lid/characteristics")
adv_bim  = torch.load("../data/extracted_characteristics/run_1/cif10/wrn28-10/bim/lid/characteristics_adv")
nor_pgd = torch.load("../data/extracted_characteristics/run_1/cif10/wrn28-10/pgd/lid/characteristics")
adv_pgd = torch.load("../data/extracted_characteristics/run_1/cif10/wrn28-10/pgd/lid/characteristics_adv")
nor_aa = torch.load("../data/extracted_characteristics/run_1/cif10/wrn28-10/aa/lid/characteristics")
adv_aa = torch.load("../data/extracted_characteristics/run_1/cif10/wrn28-10/aa/lid/characteristics_adv")
nor_df = torch.load("../data/extracted_characteristics/run_1/cif10/wrn28-10/df/lid/characteristics")
adv_df = torch.load("../data/extracted_characteristics/run_1/cif10/wrn28-10/df/lid/characteristics_adv")
nor_cw = torch.load("../data/extracted_characteristics/run_1/cif10/wrn28-10/cw/lid/characteristics")
adv_cw = torch.load("../data/extracted_characteristics/run_1/cif10/wrn28-10/cw/lid/characteristics_adv")

In [38]:
nor = [nor_fgsm, nor_bim, nor_pgd, nor_aa, nor_df, nor_cw,]
adv = [adv_fgsm, adv_bim, adv_pgd, adv_aa, adv_df, adv_cw,]

In [39]:
np.ones(nor_fgsm.shape[0]).astype('int').shape

(2000,)

In [40]:
nor_char.shape

(2000, 13)

In [50]:
factor = 0.8
nr_mean = 3
mean_tables = []
nr_samples=2000

for it in range(nr_mean):
    random_state = random.randint(100)
    print("random state: ", random_state)
    final_table = np.zeros((len(nor), len(adv)))

    for rows in range(len(nor)):
        
        X_nor = nor[rows]
        X_adv = adv[rows]
        if len(X_nor.shape) > 2: 
            X_nor = nor[rows].reshape((nor[rows].shape[0], -1))[:,::-1]
            X_adv = adv[rows].reshape((adv[rows].shape[0], -1))[:,::-1]
        
        
        y_nor = np.zeros(X_nor.shape[0]).astype('int')
        y_adv = np.ones(X_nor.shape[0]).astype('int')
        
        
        x_train_n, x_test_n, y_train_n, y_test_n = train_test_split(X_nor, y_nor, test_size=1-factor, train_size=factor, random_state=random_state)
        x_train_a, x_test_a, y_train_a, y_test_a = train_test_split(X_adv, y_adv, test_size=1-factor, train_size=factor, random_state=random_state)

        X_train = np.concatenate((x_train_n, x_train_a))
        y_train = np.concatenate((y_train_n, y_train_a))
                
        clf = RandomForestClassifier(n_estimators=300, n_jobs=-1)
        
        clf.fit(X_train, y_train)
                
                
        for cols in range(len(nor)):
            test_size = int(nr_samples * (1-factor))
            X_test_nor = nor[cols][:test_size]
            X_test_adv = adv[cols][:test_size]
            
            if len(nor[rows].shape) > 2: 
                X_test_nor = tmp_nor.reshape((tmp_nor.shape[0], -1))[:,::-1]
                X_test_adv = tmp_adv.reshape((tmp_adv.shape[0], -1))[:,::-1]
            
        
            X_test = np.concatenate((X_test_nor, X_test_adv))
            y_test = np.concatenate((y_nor[:test_size], y_adv[:test_size]))

            score_transfer = clf.score(X_test, y_test)
            final_table[rows,cols] = score_transfer
        
    mean_tables.append(final_table)

final_table

final_mean = np.mean(mean_tables, axis=0)
final_var  = np.var(mean_tables, axis=0)

random state:  12
random state:  72
random state:  92
random state:  2
random state:  69


In [51]:
final_mean

array([[0.98245614, 0.71328321, 0.74010025, 0.87042607, 0.77067669,
        0.73458647],
       [0.85062657, 0.96491228, 0.90075188, 0.96240602, 0.78245614,
        0.73609023],
       [0.87067669, 0.89423559, 0.96842105, 0.96942356, 0.8160401 ,
        0.77218045],
       [0.8518797 , 0.760401  , 0.81027569, 0.98596491, 0.82155388,
        0.75889724],
       [0.85313283, 0.76541353, 0.79598997, 0.95889724, 0.98446115,
        0.93433584],
       [0.84310777, 0.80626566, 0.84686717, 0.95864662, 0.96992481,
        0.9764411 ]])

In [52]:
final_var

array([[8.16577785e-06, 4.61052380e-05, 1.16833437e-05, 6.65824963e-06,
        1.44471454e-05, 4.23364175e-05],
       [3.16580926e-05, 8.16577785e-06, 2.91455456e-05, 2.82661541e-05,
        4.77383936e-06, 3.93213610e-05],
       [2.41204515e-05, 1.85928480e-05, 1.50752822e-06, 1.35677540e-05,
        4.17082807e-05, 1.09295796e-05],
       [1.15577163e-05, 4.42208278e-05, 4.14570260e-06, 2.76380173e-06,
        1.63315557e-06, 7.91452315e-06],
       [8.41703256e-06, 7.31151186e-05, 2.85174088e-05, 7.78889580e-06,
        6.65824963e-06, 2.88942909e-06],
       [1.03014428e-05, 3.61806773e-05, 2.16079045e-05, 6.28136758e-06,
        1.19345984e-05, 5.90448552e-06]])

In [53]:
mean_tables

[array([[0.98621554, 0.71303258, 0.7443609 , 0.87343358, 0.77568922,
         0.74561404],
        [0.84085213, 0.96867168, 0.9085213 , 0.96616541, 0.78070175,
         0.73182957],
        [0.86340852, 0.90100251, 0.96867168, 0.97243108, 0.80325815,
         0.77192982],
        [0.84962406, 0.7556391 , 0.81077694, 0.98621554, 0.82205514,
         0.7556391 ],
        [0.85714286, 0.75438596, 0.79448622, 0.95864662, 0.98496241,
         0.93483709],
        [0.84837093, 0.81077694, 0.84210526, 0.95989975, 0.97243108,
         0.97744361]]),
 array([[0.98496241, 0.70300752, 0.73558897, 0.86842105, 0.76566416,
         0.72932331],
        [0.85714286, 0.96491228, 0.89598997, 0.96365915, 0.78446115,
         0.7481203 ],
        [0.87593985, 0.89097744, 0.96867168, 0.96992481, 0.81829574,
         0.77067669],
        [0.84837093, 0.76566416, 0.81328321, 0.98370927, 0.82330827,
         0.7593985 ],
        [0.85213033, 0.75814536, 0.79197995, 0.96365915, 0.98621554,
         0.93609023

In [54]:
mean_tables[0]

array([[0.98621554, 0.71303258, 0.7443609 , 0.87343358, 0.77568922,
        0.74561404],
       [0.84085213, 0.96867168, 0.9085213 , 0.96616541, 0.78070175,
        0.73182957],
       [0.86340852, 0.90100251, 0.96867168, 0.97243108, 0.80325815,
        0.77192982],
       [0.84962406, 0.7556391 , 0.81077694, 0.98621554, 0.82205514,
        0.7556391 ],
       [0.85714286, 0.75438596, 0.79448622, 0.95864662, 0.98496241,
        0.93483709],
       [0.84837093, 0.81077694, 0.84210526, 0.95989975, 0.97243108,
        0.97744361]])